In [ ]:
import kagglehub
import numpy as np #for linear algebra
import pandas as pd #for handling data 
from matplotlib import pyplot as plt #for data visualization and tools 


# Download latest version
path = kagglehub.dataset_download("animatronbot/mnist-digit-recognizer")

print("Path to dataset files:", path)

data = pd.read_csv('/kaggle/input/mnist-digit-recognizer/train.csv')

data = np.array(data)
m, n = data.shape 
np.random.shuffle(data)
print(data)

In [ ]:
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n] #taking only n rows 
X_dev = X_dev / 255 #reducing the pixels to the level of 255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n] #taking only n rows 
X_train = X_train / 255 #reducing the pixels to the level of 255
_, m_train = X_train.shape

def ini_params():
    W1 = np.random.rand(10,784) - 0.5 #this value should be a probability value ranging from 0 to 1
    b1 = np.random.rand(10,1) - 0.5 #initial bias matrix b1
    W2 = np.random.rand(10,10) - 0.5 #initial weight W2 (for hidden layer)
    b2 = np.random.rand(10,1) - 0.5 #initial bias b2 (for hidden layer)
    return W1, b1, W2, b2

def RELU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_propagation(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = RELU(Z1)
    Z2 = W2.dot(A1) + b2 
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def RELU_der(Z):
    return Z>0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    #here the size gives the number of samples in the Y matrix and Y.max() gives the number of different classes in Y
    #we have created a matrix of zeros
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_propagation(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * RELU_der(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m *np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_par(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def get_pred(A2):
    return np.argmax(A2,0)

def get_accuracy(prediction, Y):
    print(prediction, Y)
    return np.sum(prediction == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = ini_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_propagation(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_par(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("interation: ", i)
            predictions = get_pred(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1000)

def make_pred(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_propagation(W1, b1, W2, b2, X)
    predictions = get_pred(A2)
    return predictions

def test_pred(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_pred(current_image, W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape(28,28) * 255
    plt.gray()
    plt.imshow(current_image, interpolation = "nearest")
    plt.show()

In [ ]:
indices = [0,1,2,3,4,5,6,7,8,9,10]
for i in indices:
    test_pred(i, W1, b1, W2, b2)